In [1]:
! pip install haystack-ai==2.18
! pip install "sentence-transformers>=4.1.0"
! pip install ollama-haystack

Defaulting to user installation because normal site-packages is not writeable
  Using cached haystack_ai-2.18.0-py3-none-any.whl.metadata (15 kB)
Using cached haystack_ai-2.18.0-py3-none-any.whl (591 kB)
  Attempting uninstall: haystack-ai
    Found existing installation: haystack-ai 2.20.0
    Uninstalling haystack-ai-2.20.0:
      Successfully uninstalled haystack-ai-2.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ollama-haystack 5.3.0 requires haystack-ai>=2.19.0, but you have haystack-ai 2.18.0 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached haystack_ai-2.20.0-py3-none-any.whl.metadata (15 kB)
Using cached haystack_ai-2.20.0-py3-none-any.whl (624 kB)
  Attempting uninstall: haystack-ai
    Found existing in

In [2]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders.chat_prompt_builder import ChatPromptBuilder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.dataclasses import ChatMessage
from typing import List, Dict, Any
from haystack import Pipeline
from haystack import Document
from haystack import component
from pathlib import Path
import re

/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing a document store and adding to InMemoryDocumentStore

In [3]:
# Initialize document store
document_store = InMemoryDocumentStore()

# Load and embed documents
file_path = "Bpmn_code_and_Description.txt"
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

docs = [Document(content=text)]
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
doc_embedder.warm_up()
embedded_docs = doc_embedder.run(docs)
document_store.write_documents(embedded_docs["documents"])

retriever = InMemoryEmbeddingRetriever(document_store=document_store)
text_embedder = SentenceTransformersTextEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2")


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.76it/s]


Custom component

In [4]:
# Custom component to convert chat messages to string
@component
class ChatMessagesToString:
    @component.output_types(prompt=str)
    def run(self, messages: List[ChatMessage]) -> Dict[str, Any]:
        prompt = ""
        for msg in messages:
            if msg.role == "system":
                prompt += f"<|start_header_id|>system<|end_header_id|>\n{msg.text}<|eot_id|>\n"
            elif msg.role == "user":
                prompt += f"<|start_header_id|>user<|end_header_id|>\n{msg.text}<|eot_id|>\n"
        prompt += "<|start_header_id|>assistant<|end_header_id|>\n"
        return {"prompt": prompt}

# Custom component to extract example from retrieved documents
@component
class ExtractExample:
    @component.output_types(example=str)
    def run(self, documents: List[Document]) -> Dict[str, Any]:
        if documents and len(documents) > 0:
            # Return the content of the first retrieved document as example
            return {"example": documents[0].content}
        return {"example": "No example available."}

# Custom component to collect generator outputs
@component
class CollectGeneratorOutputs:
    @component.output_types(
        chat_output=str,
        with_doc_output=str,
        without_examples_output=str
    )
    def run(
        self,
        chat_replies: List[str],
        with_doc_replies: List[str],
        without_examples_replies: List[str]
    ) -> Dict[str, Any]:
        def clean(r): 
            return r[0].strip() if r and len(r) > 0 else "No output generated"
        
        return {
            "chat_output": clean(chat_replies),
            "with_doc_output": clean(with_doc_replies),
            "without_examples_output": clean(without_examples_replies)
        }

ChatPromptBuilder

In [5]:
# 1. ChatPromptBuilder (role-based, uses real ChatMessage)
chat_template = [
    ChatMessage.from_system(
        "You are an expert in describing BPMN process modeles. "
        "Describe the BPMN model in natural language. "
        "Mention every task, gateway, and event. "
        "Example:\n{{example}}"
    ),
    ChatMessage.from_user("Describe this BPMN model:\n{{bpmn_code}}")
]
chat_prompt_builder = ChatPromptBuilder(
    template=chat_template,
    required_variables=["bpmn_code", "example"]
)

# 2. Regular PromptBuilder with example
prompt_with_example = PromptBuilder(
    template="""
You are an expert in describing BPMN process modeles.
Describe the BPMN model in natural language.
Mention every task, gateway, and event.

Example:
{{example}}

BPMN Model:
{{bpmn_code}}

Description:
""",
    required_variables=["bpmn_code", "example"]
)

# 3. Regular PromptBuilder without example
prompt_no_example = PromptBuilder(
    template="""
You are an expert in describing BPMN process modeles.
Describe the BPMN model in natural language.
Mention every task, gateway, and event.

BPMN Model:
{{bpmn_code}}

Description:
""",
    required_variables=["bpmn_code"]
)

Initialize generators

In [6]:

chat_gen = OllamaGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=1800,
    generation_kwargs={"num_ctx": 4096, "temperature": 0.9}
)

gen_with_example = OllamaGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=1800,
    generation_kwargs={"num_ctx": 4096, "temperature": 0.9}
)
gen_no_example = OllamaGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=1800,
    generation_kwargs={"num_ctx": 4096, "temperature": 0.9}
)

Function to read a BPMN file and remove the coordinates automatically for the query

In [7]:
def load_bpmn_without_coordinates(path_to_file: str) -> str:
    with open(path_to_file, "r", encoding="utf-8") as f:
        data = f.read()
    data = re.sub(r"<dc:Bounds[^>]*/>", "", data)
    data = re.sub(r"<(?:di|omgdi):waypoint\s*[^>]*/>", "", data)  # fixed
    data = re.sub(r'\s(x|y|width|height)="[^"]*"', "", data)
    data = "\n".join(line for line in data.splitlines() if line.strip())
    return data

Build pipeline

In [8]:
pipe = Pipeline()

# Add components
pipe.add_component("text_embedder", text_embedder)
pipe.add_component("retriever", retriever)
pipe.add_component("extract_example", ExtractExample())  # New component to extract example
pipe.add_component("chat_prompt_builder", chat_prompt_builder)
pipe.add_component("chat_to_string", ChatMessagesToString())
pipe.add_component("chat_gen", chat_gen)
pipe.add_component("prompt_with_example", prompt_with_example)
pipe.add_component("gen_with_example", gen_with_example)
pipe.add_component("prompt_no_example", prompt_no_example)
pipe.add_component("gen_no_example", gen_no_example)
pipe.add_component("collector", CollectGeneratorOutputs())


Connect all branches

In [9]:
pipe.connect("text_embedder.embedding", "retriever.query_embedding")
pipe.connect("retriever.documents", "extract_example.documents")  # Connect to extract_example
pipe.connect("extract_example.example", "chat_prompt_builder.example")  # Connect example to chat prompt
pipe.connect("extract_example.example", "prompt_with_example.example")  # Connect example to regular prompt
pipe.connect("chat_prompt_builder.prompt", "chat_to_string.messages")
pipe.connect("chat_to_string.prompt", "chat_gen.prompt")
pipe.connect("prompt_with_example.prompt", "gen_with_example.prompt")
pipe.connect("prompt_no_example.prompt", "gen_no_example.prompt")
pipe.connect("chat_gen.replies", "collector.chat_replies")
pipe.connect("gen_with_example.replies", "collector.with_doc_replies")
pipe.connect("gen_no_example.replies", "collector.without_examples_replies")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - extract_example: ExtractExample
  - chat_prompt_builder: ChatPromptBuilder
  - chat_to_string: ChatMessagesToString
  - chat_gen: OllamaGenerator
  - prompt_with_example: PromptBuilder
  - gen_with_example: OllamaGenerator
  - prompt_no_example: PromptBuilder
  - gen_no_example: OllamaGenerator
  - collector: CollectGeneratorOutputs
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (list[float])
  - retriever.documents -> extract_example.documents (list[Document])
  - extract_example.example -> chat_prompt_builder.example (str)
  - extract_example.example -> prompt_with_example.example (str)
  - chat_prompt_builder.prompt -> chat_to_string.messages (list[ChatMessage])
  - chat_to_string.prompt -> chat_gen.prompt (str)
  - chat_gen.replies -> collector.chat_replies (List[str])
  - prompt_with_example.prompt -> gen_with_example.prompt (str)
  - gen_with_exa

In [11]:
# Load BPMN file
bpmn_file = "passenger_check_in_counter.bpmn"  
bpmn_code = load_bpmn_without_coordinates(bpmn_file)

# Run pipeline
result = pipe.run({
    "text_embedder": {"text": bpmn_code},
    "chat_prompt_builder": {"bpmn_code": bpmn_code},
    "prompt_with_example": {"bpmn_code": bpmn_code},
    "prompt_no_example": {"bpmn_code": bpmn_code}
})

# Output results
print("=== Chat (Role-Based) ===")
print(result["collector"]["chat_output"])
print("\n" + "="*50 + "\n")
print("=== With Example ===")
print(result["collector"]["with_doc_output"])
print("\n" + "="*50 + "\n")

print("=== No Example ===")
print(result["collector"]["without_examples_output"])

Batches: 100%|██████████| 1/1 [00:00<00:00, 38.99it/s]


=== Chat (Role-Based) ===
This BPMN (Business Process Model and Notation) diagram describes a business process for airport check-in. Here's a breakdown of the process:

**Process Overview**

The process starts with a **Start Event**, which is the beginning of the process. The process then flows through several activities, including checking in, luggage handling, security screening, and finally ends with an **End Event**, which marks the completion of the process.

**Main Activities**

1. **Check-in**: The first activity is to check-in at the airport. This is represented by the activity "Activity_0f1w87w" in the diagram.
2. **Luggage Handling**: After checking in, the passenger must handle their luggage. This is represented by the activity "Activity_074g467".
3. **Security Screening**: If the passenger has luggage, they will go through security screening. This is represented by the activity "Activity_09u2bux".
4. **Boarding Pass Distribution**: After security screening, the passenger re

In [12]:
bpmn_file2 = "passenger_check_in_machine.bpmn"
bpmn_code2 = load_bpmn_without_coordinates(bpmn_file2)

# Run pipeline
result2 = pipe.run({
    "text_embedder": {"text": bpmn_code2},
    "chat_prompt_builder": {"bpmn_code": bpmn_code2},
    "prompt_with_example": {"bpmn_code": bpmn_code2},
    "prompt_no_example": {"bpmn_code": bpmn_code2}
})

# Output results
print("=== Chat (Role-Based) ===")
print(result2["collector"]["chat_output"])
print("\n" + "="*50 + "\n")
print("=== With Example ===")
print(result2["collector"]["with_doc_output"])
print("\n" + "="*50 + "\n")

print("=== No Example ===")
print(result2["collector"]["without_examples_output"])

Batches: 100%|██████████| 1/1 [00:00<00:00, 39.35it/s]


=== Chat (Role-Based) ===
This BPMN (Business Process Model and Notation) diagram represents a process for passenger check-in at an airport. Here's a breakdown of the process:

**Process Overview**

The process starts with the "Start Event" (1), which initiates the process. The first step is the "Passenger gets Boarding Pass" (9), where the passenger receives their boarding pass.

Next, the process reaches a decision point at the "Gateway: Does the passenger have baggage?" (7). If the answer is yes, the process flows to the "Passenger goes to baggage claim counter" (14).

If the answer is no, the process branches out to two parallel paths:

* The first path involves the "Passenger gets Boarding Pass" (9) step again.
* The second path involves a decision point at the "Gateway: Does the passenger have baggage?" (7). If the answer is yes, the process flows to the "Passenger goes to baggage claim counter" (14). If the answer is no, the process ends with the "End Event" (16).

**Decision Po

In [13]:
bpmn_file3 = "baggage_departure.bpmn"
bpmn_code3 = load_bpmn_without_coordinates(bpmn_file3)

# Run pipeline
result3 = pipe.run({
    "text_embedder": {"text": bpmn_code3},
    "chat_prompt_builder": {"bpmn_code": bpmn_code3},
    "prompt_with_example": {"bpmn_code": bpmn_code3},
    "prompt_no_example": {"bpmn_code": bpmn_code3}
})

# Output results
print("=== Chat (Role-Based) ===")
print(result3["collector"]["chat_output"])
print("\n" + "="*50 + "\n")
print("=== With Example ===")
print(result3["collector"]["with_doc_output"])
print("\n" + "="*50 + "\n")

print("=== No Example ===")
print(result3["collector"]["without_examples_output"])

Batches: 100%|██████████| 1/1 [00:00<00:00, 35.47it/s]


=== Chat (Role-Based) ===
This is a BPMN (Business Process Model and Notation) file, which represents a business process as a flowchart. Here's an overview of the process:

**Start Event**

The process starts with a Start Event, labeled "Process" with a key "_gjrx3e".

**Sequence Flow 1: Arrival Check**

The first sequence flow is from the Start Event to an Activity named "Arrival Check" (Activity_06gocr1). This activity likely checks whether the passenger has arrived at the designated location.

**Sequence Flow 2: Baggage Drop-off**

From the Arrival Check activity, a sequence flow goes to another activity named "Baggage Drop-off" (Activity_0ah4rhc).

**Gateways and Sequence Flows**

The process then branches out through several gateways:

1. An Exclusive OR Gateway (XOR) labeled "Is Baggage Missing?" splits the process into two paths:
	* If baggage is missing, a sequence flow goes to an Activity named "Notify Passenger" (Activity_1hbwzez).
	* If baggage is present, a sequence flow go

In [14]:

def evaluate_precision_recall(bpmn_code: str, generated_text: str) -> dict:
    # Extract element IDs using robust, case-insensitive regex
    task_ids = set(re.findall(r'<bpmn:task\s+[^>]*?id\s*=\s*["\']([^"\']+)["\']', bpmn_code, re.IGNORECASE))
    gateway_ids = set(re.findall(r'<bpmn:(?:exclusive|inclusive|parallel)gateway\s+[^>]*?id\s*=\s*["\']([^"\']+)["\']', bpmn_code, re.IGNORECASE))
    event_ids = set(re.findall(r'<bpmn:(?:start|end|intermediate)event\s+[^>]*?id\s*=\s*["\']([^"\']+)["\']', bpmn_code, re.IGNORECASE))

    all_elements = task_ids | gateway_ids | event_ids

    if not all_elements:
        return {
            "precision": 0.0, "recall": 0.0,
            "found_elements": 0, "total_elements": 0,
            "tasks_found": 0, "gateways_found": 0, "events_found": 0,
            "total_tasks": 0, "total_gateways": 0, "total_events": 0,
            "missed_elements": 0
        }

    found_tasks = sum(1 for tid in task_ids if tid in generated_text)
    found_gateways = sum(1 for gid in gateway_ids if gid in generated_text)
    found_events = sum(1 for eid in event_ids if eid in generated_text)

    found_elements = found_tasks + found_gateways + found_events
    total_elements = len(all_elements)
    recall = found_elements / total_elements if total_elements > 0 else 0.0
    precision = recall  # coverage-focused metric

    return {
        "precision": precision,
        "recall": recall,
        "found_elements": found_elements,
        "total_elements": total_elements,
        "missed_elements": total_elements - found_elements,
        "tasks_found": found_tasks,
        "gateways_found": found_gateways,
        "events_found": found_events,
        "total_tasks": len(task_ids),
        "total_gateways": len(gateway_ids),
        "total_events": len(event_ids)
    }

generator_output_key = "with_doc_output"

for file_name, pipe_result in [
    ("passenger_check_in_counter.bpmn", result),
    ("passenger_check_in_machine.bpmn", result2),
    ("baggage_departure.bpmn", result3)
]:
    print(f"\n--- {file_name} ---")
    bpmn_clean = load_bpmn_without_coordinates(file_name)
    generated_desc = pipe_result["collector"][generator_output_key]
    
    scores = evaluate_precision_recall(bpmn_clean, generated_desc)
    
    print(f"Precision: {scores['precision']:.2f}")
    print(f"Recall   : {scores['recall']:.2f}")
    print(f"Coverage : {scores['found_elements']}/{scores['total_elements']} elements mentioned")
    print(f"  • Tasks     : {scores['tasks_found']}/{scores['total_tasks']}")
    print(f"  • Gateways  : {scores['gateways_found']}/{scores['total_gateways']}")
    print(f"  • Events    : {scores['events_found']}/{scores['total_events']}")


--- passenger_check_in_counter.bpmn ---
Precision: 0.00
Recall   : 0.00
Coverage : 0/11 elements mentioned
  • Tasks     : 0/7
  • Gateways  : 0/2
  • Events    : 0/2

--- passenger_check_in_machine.bpmn ---
Precision: 0.00
Recall   : 0.00
Coverage : 0/11 elements mentioned
  • Tasks     : 0/7
  • Gateways  : 0/2
  • Events    : 0/2

--- baggage_departure.bpmn ---
Precision: 0.22
Recall   : 0.22
Coverage : 4/18 elements mentioned
  • Tasks     : 0/10
  • Gateways  : 3/5
  • Events    : 1/3


In [15]:

def evaluate_precision_recall(bpmn_code: str, generated_text: str) -> dict:
    # Extract element IDs using robust, case-insensitive regex
    task_ids = set(re.findall(r'<bpmn:task\s+[^>]*?id\s*=\s*["\']([^"\']+)["\']', bpmn_code, re.IGNORECASE))
    gateway_ids = set(re.findall(r'<bpmn:(?:exclusive|inclusive|parallel)gateway\s+[^>]*?id\s*=\s*["\']([^"\']+)["\']', bpmn_code, re.IGNORECASE))
    event_ids = set(re.findall(r'<bpmn:(?:start|end|intermediate)event\s+[^>]*?id\s*=\s*["\']([^"\']+)["\']', bpmn_code, re.IGNORECASE))

    all_elements = task_ids | gateway_ids | event_ids

    if not all_elements:
        return {
            "precision": 0.0, "recall": 0.0,
            "found_elements": 0, "total_elements": 0,
            "tasks_found": 0, "gateways_found": 0, "events_found": 0,
            "total_tasks": 0, "total_gateways": 0, "total_events": 0,
            "missed_elements": 0
        }

    found_tasks = sum(1 for tid in task_ids if tid in generated_text)
    found_gateways = sum(1 for gid in gateway_ids if gid in generated_text)
    found_events = sum(1 for eid in event_ids if eid in generated_text)

    found_elements = found_tasks + found_gateways + found_events
    total_elements = len(all_elements)
    recall = found_elements / total_elements if total_elements > 0 else 0.0
    precision = recall  # coverage-focused metric

    return {
        "precision": precision,
        "recall": recall,
        "found_elements": found_elements,
        "total_elements": total_elements,
        "missed_elements": total_elements - found_elements,
        "tasks_found": found_tasks,
        "gateways_found": found_gateways,
        "events_found": found_events,
        "total_tasks": len(task_ids),
        "total_gateways": len(gateway_ids),
        "total_events": len(event_ids)
    }

generator_output_key = "without_examples_output"

for file_name, pipe_result in [
    ("passenger_check_in_counter.bpmn", result),
    ("passenger_check_in_machine.bpmn", result2),
    ("baggage_departure.bpmn", result3)
]:
    print(f"\n--- {file_name} ---")
    bpmn_clean = load_bpmn_without_coordinates(file_name)
    generated_desc = pipe_result["collector"][generator_output_key]
    
    scores = evaluate_precision_recall(bpmn_clean, generated_desc)
    
    print(f"Precision: {scores['precision']:.2f}")
    print(f"Recall   : {scores['recall']:.2f}")
    print(f"Coverage : {scores['found_elements']}/{scores['total_elements']} elements mentioned")
    print(f"  • Tasks     : {scores['tasks_found']}/{scores['total_tasks']}")
    print(f"  • Gateways  : {scores['gateways_found']}/{scores['total_gateways']}")
    print(f"  • Events    : {scores['events_found']}/{scores['total_events']}")


--- passenger_check_in_counter.bpmn ---
Precision: 0.09
Recall   : 0.09
Coverage : 1/11 elements mentioned
  • Tasks     : 0/7
  • Gateways  : 0/2
  • Events    : 1/2

--- passenger_check_in_machine.bpmn ---
Precision: 0.82
Recall   : 0.82
Coverage : 9/11 elements mentioned
  • Tasks     : 7/7
  • Gateways  : 2/2
  • Events    : 0/2

--- baggage_departure.bpmn ---
Precision: 0.28
Recall   : 0.28
Coverage : 5/18 elements mentioned
  • Tasks     : 2/10
  • Gateways  : 2/5
  • Events    : 1/3
